# Stability Verification for an Inverted Pendulum

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from __future__ import division, print_function

import numpy as np
import tensorflow as tf
import gpflow
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors as mpl_colors
%matplotlib inline

import time

import safe_learning
from utilities import InvertedPendulum, debug

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x: x
    
np_dtype = safe_learning.config.np_dtype
tf_dtype = safe_learning.config.dtype

try:
    session.close()
except NameError:
    pass
session = tf.InteractiveSession()

# TODO debug flags ***************************************#

import pandas
pandas.options.display.float_format = '{:,.4f}'.format
pandas.set_option('expand_frame_repr', False)
np.set_printoptions(precision=4)

# Saturate the action so that it lies in [-1, 1]
SATURATE = True

# Use the true physical parameters in the GP model
USE_TRUE_PARAMETERS = False

# Use the linearized discrete-time model as the true underlying dynamics
USE_LINEAR_DYNAMICS = False

# Use a threshold of zero when checking for stability
USE_ZERO_THRESHOLD = False

#
USE_LINEAR_KERNELS = False

#
USE_LIPSCHITZ_SCALING = True

# Scaling factor for GP confidence intervals
BETA = 2.

#
TRAIN_HYPERPARAMETERS = False

#
GP_SCALING = 1.

#
NOISE_VAR = 0.001 ** 2

#******************************************************#


In [ ]:
def local_discretization(center, tau, N):
    offset = (1 - 1 / N) * tau / 2
    lower = center - offset
    upper = center + offset
    if len(N) > 1:
        discrete_points = [np.linspace(low, up, n, dtype=config.np_dtype)
                           for low, up, n in zip(lower, upper, N)]
    else:
        discrete_points = [np.linspace(low, up, N[0], dtype=config.np_dtype)
                           for low, up in zip(lower, upper)]
    mesh = np.meshgrid(*discrete_points, indexing='ij')
    points = np.column_stack(col.ravel() for col in mesh)
    local_points = points.astype(config.np_dtype)
    return local_points


class MultiResolutionGrid(safe_learning.GridWorld):
    """ """
    def __init__(self, limits, num_points, N=None):
        """ """
        super(MultiResolutionGrid, self).__init__(limits, num_points)
        if N is not None:
            self._N = N
        else:
            self._N = np.ones(nindex, dtype=np.int)
        self._refined_points = None
        self._refined_nindex = np.sum(self._N)
    
    @property
    def refined_points(self):
        """ """
        if self._refined_points is None:
            self.update_resolution(self._N)
        return self._refined_points
    
    def update_resolution(self, N):
        """ """
        dim = int(self.discretization.ndim)
        tau = self.discretization.unit_maxes

        # TODO different N values per state?
        gridify = lambda point: local_discretization(point, tau, N)
        local_grids = np.apply_along_axis(gridify, 1, self._all_points)

        self._N = N
        self._refined_points = local_grids.reshape((-1, dim))
        self._refined_nindex = np.sum(self._N)

## Dynamics

In [ ]:
# Constants
dt = 0.01   # sampling time
g = 9.81    # gravity

# True system parameters
m = 0.15    # pendulum mass
L = 0.5     # pole length
b = 0.1     # rotational friction

# State and action normalizers
theta_max = np.deg2rad(30)
omega_max = np.sqrt(g / L)
u_max = g * m * L * np.sin(theta_max)

state_norm = (theta_max, omega_max)
action_norm = (u_max, )

# Constraints for initial 'safe' states
theta_safe = np.deg2rad(8)
omega_safe = 0.5*np.sqrt(g / L)

# Dimensions and domains
state_dim = 2
action_dim = 1
state_limits = np.array([[-1., 1.]]*state_dim)
action_limits = np.array([[-1., 1.]]*action_dim)

print(np.rad2deg(state_norm))

# True system
true_pendulum = InvertedPendulum(m, L, b, dt, [state_norm, action_norm])
A_true, B_true = true_pendulum.linearize()

if USE_LINEAR_DYNAMICS:
    true_dynamics = safe_learning.functions.LinearSystem((A_true, B_true), name='true_dynamics')
else:
    true_dynamics = true_pendulum.__call__

# "Wrong" system
m = 0.1     # pendulum mass
L = 0.4     # pole length
b = 0.0     # rotational friction
pendulum = InvertedPendulum(m, L, b, dt, [state_norm, action_norm])
A, B = pendulum.linearize()

if USE_TRUE_PARAMETERS:
    A = A_true
    B = B_true
mean_dynamics = safe_learning.LinearSystem((A, B), name='mean_dynamics')

## GP Model

In [ ]:
m_true = np.hstack((A_true, B_true))
m = np.hstack((A, B))
variances = (m_true - m) ** 2

# Make sure at least some non-zero prior variance is maintained
np.clip(variances, 1e-3, None, out=variances)

# Measurement noise
noise_var = NOISE_VAR

# Input to GP is of the form (x,u)
full_dim = state_dim + action_dim

# Kernels
if USE_LINEAR_KERNELS:
    kernel_theta = gpflow.kernels.Linear(full_dim, variance=variances[0, :], ARD=True)

    kernel_omega = gpflow.kernels.Linear(full_dim, variance=variances[1, :], ARD=True)

else:
    kernel_theta = (gpflow.kernels.Linear(full_dim, variance=variances[0, :], ARD=True)
                    + gpflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
                    * gpflow.kernels.Linear(1, variance=variances[0, 1]))

    kernel_omega = (gpflow.kernels.Linear(full_dim, variance=variances[1, :], ARD=True)
                    + gpflow.kernels.Matern32(1, lengthscales=1, active_dims=[0])
                    * gpflow.kernels.Linear(1, variance=variances[1, 1]))

# Mean dynamics
mean_function_theta = safe_learning.LinearSystem((A[[0], :], B[[0], :]), name='mean_dynamics_theta')
mean_function_omega = safe_learning.LinearSystem((A[[1], :], B[[1], :]), name='mean_dynamics_omega')

# Define a GP model over the dynamics
gp_theta = safe_learning.GPRCached(np.empty((0, full_dim), dtype=np_dtype),
                                   np.empty((0, 1), dtype=np_dtype),
                                   kernel_theta,
                                   mean_function_theta)

gp_omega = safe_learning.GPRCached(np.empty((0, full_dim), dtype=np_dtype),
                                   np.empty((0, 1), dtype=np_dtype),
                                   kernel_omega,
                                   mean_function_omega)

gp_theta.likelihood.variance = noise_var
gp_omega.likelihood.variance = noise_var

#
if TRAIN_HYPERPARAMETERS:
    # Sample state-actions (x, u) and observations from the dynamics
    N = 1e2
    states = sample_box(state_limits, N)
    actions = sample_box(action_limits, N)
    X = np.concatenate((states, actions), axis=1)
    Y = true_dynamics(states, actions).eval()

    for i, gp in enumerate((gp_theta, gp_omega)):

        # 
        print('Original parameters:', gp, '\n')

        # Optimize the parameters of each GP via MLE
        print('Optimizing ...\n')
        gp.X = X
        gp.Y = Y[:, i].reshape((-1, 1))
        gp.optimize()

        # Reset GP data matrices for safe learning
        gp.X = np.empty((0, full_dim), dtype=np_dtype)
        gp.Y = np.empty((0, 1), dtype=np_dtype)
        gp.update_cache()

        # 
        print('New parameters:', gp, '\n')

#
gp_theta_fun = safe_learning.GaussianProcess(gp_theta, BETA)
gp_omega_fun = safe_learning.GaussianProcess(gp_omega, BETA)

# Stack GP functions => block-diagonal kernel matrix
dynamics = safe_learning.FunctionStack((gp_theta_fun, gp_omega_fun))

## State Discretization

In [ ]:
# Number of states along each dimension
num_states = [201, 201]
# num_states = [2001, 2001]

# State grid
grid_limits = np.array([[-1., 1.], [-1., 1.]])
state_discretization = safe_learning.GridWorld(grid_limits, num_states)

# Discretization constant
if USE_ZERO_THRESHOLD:
    tau = 0.0
else:
    tau = np.sum(state_discretization.unit_maxes) / 2

print('Grid size: {}'.format(state_discretization.nindex))
print('Discretization constant: {}'.format(tau))

## Cost Function

In [ ]:
# State cost matrix
Q = np.diag([1., 2.])

# Action cost matrix
R = 1.2*np.identity(action_dim)

# Normalize cost matrices
cost_norm = np.amax([Q.max(), R.max()])
Q = Q / cost_norm
R = R / cost_norm

# Quadratic cost function
cost_function = safe_learning.QuadraticFunction(block_diag(Q, R), name='cost_function')

## Policy

In [ ]:
# Fix policy to the LQR solution for the true system
K, P = safe_learning.utilities.dlqr(A_true, B_true, Q, R)
policy = safe_learning.LinearSystem(-K, name='policy')

if SATURATE:
    policy = safe_learning.Saturation(policy, -1, 1)
    
# TensorFlow variables
tf_states = tf.placeholder(tf_dtype, shape=[None, state_dim], name='states')
tf_actions = policy(tf_states)

## Lyapunov Function

In [ ]:
# Define the Lyapunov function corresponding to the known policy
lyapunov_function = safe_learning.QuadraticFunction(P)
grad_lyapunov_function = safe_learning.LinearSystem((2*P,))

# Lipschitz constants
L_pol = lambda s: tf.constant(np.linalg.norm(-K, 1), dtype=tf_dtype)
L_dyn = lambda s: np.linalg.norm(A_true, 1) + np.linalg.norm(B_true, 1)*L_pol(s)

if USE_LIPSCHITZ_SCALING:
    L_v = lambda s: tf.abs(grad_lyapunov_function(s))
else:
    L_v = lambda s: tf.norm(grad_lyapunov_function(s), ord=1, axis=1, keep_dims=True)
    
# Set initial safe set as a level set of the Lyapunov function
values = session.run(lyapunov_function(tf_states), {tf_states: state_discretization.all_points})
cutoff = 1e-2 * np.max(values)
initial_safe_set = np.squeeze(values, axis=1) <= cutoff

# Set initial safe set as a hypercube in the state space
# safe_norm = np.array([[theta_safe / theta_max, omega_safe / omega_max]])
# norm_states = state_discretization.all_points / safe_norm
# initial_safe_set = np.all(np.logical_and(norm_states >= -1, norm_states <= 1), axis=1, keepdims=False)

# Initialize class
lyapunov = safe_learning.Lyapunov(state_discretization, lyapunov_function, dynamics, 
                                  L_dyn, L_v, tau, policy, initial_safe_set)

In [ ]:
def plot_policy(lyapunov, tf_states, state_norm=None, show=True):
    fig, ax = plt.subplots(1, 1, figsize=(6, 5), dpi=100)
#     fig.subplots_adjust(wspace=0.4, hspace=0.2)
    cmap = plt.get_cmap('viridis')
    cmap.set_under('indigo')
    cmap.set_over('gold')
    ticks = np.linspace(-1., 1., 9)
    cutoff = 1. - 1e-10
    
    if state_norm is not None:
        theta_max, omega_max = state_norm
        scale = np.array([np.rad2deg(theta_max), np.rad2deg(omega_max)]).reshape((-1, 1))
        limits = scale * lyapunov.discretization.limits
    else:
        limits = lyapunov.discretization.limits
    
    z = session.run(lyapunov.policy(tf_states), feed_dict={tf_states: lyapunov.discretization.all_points})
    z = z.reshape(lyapunov.discretization.num_points)
    im = ax.imshow(z.T, 
                   origin='lower', 
                   extent=limits.ravel(), 
                   aspect=limits[0, 0] / limits[1, 0],
                   cmap=cmap,
                   vmin=-cutoff,
                   vmax=cutoff)
    cbar = fig.colorbar(im, ax=ax, label=r'$u = \pi(x)$', ticks=ticks)
    ax.set_xlabel(r'$\theta$ [deg]')
    ax.set_ylabel(r'$\omega$ [deg/s]')
    
    if show:
        plt.show()

# Visualize policy
plot_policy(lyapunov, tf_states, state_norm)

## Initial Safe Set Visualization

In [ ]:
# Compare safe set before and after checking the decrease condition for the first time
c_max = lyapunov.feed_dict[lyapunov.c_max]
init_safe_set_size = np.sum(lyapunov.safe_set)

print('Before update ...')
print('c_max: {}'.format(c_max))
print('Safe set size: {}\n'.format(init_safe_set_size))

old_safe_set = np.copy(lyapunov.safe_set)
lyapunov.update_safe_set()

c_max = lyapunov.feed_dict[lyapunov.c_max]
init_safe_set_size = np.sum(lyapunov.safe_set)

print('After update ...')
print('c_max: {}'.format(c_max))
print('Safe set size: {}'.format(init_safe_set_size))

debug(lyapunov, true_dynamics, state_norm, plot='pendulum')

In [ ]:
# debug(lyapunov, true_dynamics, state_norm, do_print=True, newly_safe_only=False)

## Online Learning and Exploration

In [ ]:
# action_variation = np.array([-0.01, -0.001, 0.0, 0.001, 0.01], dtype=np_dtype).reshape((-1, 1))
action_variation = np.array([[0.]], dtype=np_dtype)

with tf.name_scope('add_new_measurement'):
    full_dim = state_dim + action_dim 
    tf_max_state_action = tf.placeholder(tf_dtype, shape=[1, full_dim])
    tf_measurement = true_dynamics(tf_max_state_action)
    
def update_gp():
    """Update the GP model based on an actively selected data point."""
    
    # Get a new sample location
    max_state_action, _ = safe_learning.get_safe_sample(lyapunov,
                                                        action_variation,
                                                        action_limits,
                                                        positive=False,
                                                        num_samples=1000)
    
    # Obtain a measurement of the true dynamics
    lyapunov.feed_dict[tf_max_state_action] = max_state_action
    measurement = tf_measurement.eval(feed_dict=lyapunov.feed_dict)
    
    # Add the measurement to our GP dynamics
    lyapunov.dynamics.add_data_point(max_state_action, measurement)

In [ ]:
data_per_update = 10
safe_set_updates = 3

if 'level' not in globals():
    level = np.zeros(safe_set_updates + 1)
    safe_size = np.zeros(safe_set_updates + 1)
    level[0] = lyapunov.feed_dict[lyapunov.c_max]
    safe_size[0] = np.sum(lyapunov.safe_set)
    num_data = data_per_update*np.arange(safe_set_updates + 1)
    e = 0
else:
    e = len(level) - 1
    level = np.concatenate((level, np.zeros(safe_set_updates)))
    safe_size = np.concatenate((safe_size, np.zeros(safe_set_updates)))
    temp = data_per_update*np.arange(1, safe_set_updates + 1) + num_data[-1]
    num_data = np.concatenate((num_data, temp))

start = time.time()
    
for i in range(safe_set_updates):
    
    print('Iteration {} with c_max: {}'.format(e + i + 1, lyapunov.feed_dict[lyapunov.c_max]))
    old_safe_set = np.copy(lyapunov.safe_set)

#     for _ in tqdm(range(data_per_update)):
    for _ in range(data_per_update):    
        update_gp()

    lyapunov.update_safe_set(batch_size=1000, adaptive=True, Nmax=8)
#     lyapunov.update_safe_set(batch_size=10000, adaptive=False)
    level[e + i + 1] = lyapunov.feed_dict[lyapunov.c_max]
    safe_size[e + i + 1] = np.sum(lyapunov.safe_set)
    
    current_safe_set_size = np.sum(lyapunov.safe_set)
    data = lyapunov.dynamics.functions[0].X
    print('Discretization size: {}'.format(state_discretization.all_points.shape[0]))
    print('Safe set size: {}'.format(current_safe_set_size))
    print('Growth: {}'.format(current_safe_set_size - init_safe_set_size))
    print("Data points collected: {}".format(data.shape[0]))
    print("New c_max: {}\n".format(lyapunov.feed_dict[lyapunov.c_max]))

end = time.time()
print('Duration:', end - start)

debug(lyapunov, true_dynamics, state_norm, plot='pendulum')

In [ ]:
idx = np.argmax(num_data > 200)
idx = 0
if idx == 0:
    idx = len(num_data)
print(level, '\n')
print(safe_size)
    
fig, ax = plt.subplots(1, 2, sharex=False, figsize=(10, 3), dpi=200)
fig.subplots_adjust(wspace=0.3, hspace=0.2)

ax[0].step(num_data[:idx], level[:idx], 'o--', where='post')
ax[0].set_xlabel(r'Number of data points collected', fontsize=12)
ax[0].set_ylabel(r'$c_{max}$', fontsize=12)

ax[1].step(num_data[:idx], safe_size[:idx], 'o--', where='post')
ax[1].set_xlabel(r'Number of data points collected', fontsize=12)
ax[1].set_ylabel(r'Safe set size', fontsize=12)

plt.show()

In [ ]:
# Accepted "safe" state-action pairs
# data = lyapunov.dynamics.functions[0].X
# print("Data points collected: {}\n".format(data.shape[0]))
# print(data)

In [ ]:
# print(gp_theta.predict_f(data[0,:].reshape(1, -1)))
# print(gp_theta.likelihood.variance.value)

In [ ]:
debug(lyapunov, true_dynamics, state_norm, do_print=True, newly_safe_only=True)

In [ ]:
print(lyapunov._N.shape)
print(lyapunov._N.sum())